In [3]:
import json
import random
import os
import pandas as pd
import re
import aiohttp
import asyncio
import backoff
import nest_asyncio
import time
from SPARQLWrapper import SPARQLWrapper, JSON
from requests import get
import numpy as np
import requests
from aiohttp import ClientResponseError
import logging
from tqdm import tqdm
import json

In [4]:
import gzip  # For reading compressed files
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
import pandas as pd

file_path = "./data/yago-wd-full-types.nt.gz"

try:
    with gzip.open(file_path, 'rb') as f:
        chunk = []
        
        # Use tqdm to wrap the file iterator to track progress
        for line in tqdm(f, total=5394049):
            line_str = line.decode('utf-8').strip()
            chunk.append(line_str)
            
        
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except Exception as e:
    print(f"Error processing RDF data: {e}")

100%|██████████| 5394049/5394049 [00:26<00:00, 200000.17it/s]


In [8]:
##################################
# EXECUTE FROM THIS FOR THE MAPPING
##################################à

json_file_path = "./data/mapping.json"

try:
    # Open the JSON file for reading
    with open(json_file_path, 'r') as json_file:
        mapping = json.load(json_file)
        
except FileNotFoundError:
    print(f"Error: File '{json_file_path}' not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON data: {e}")
except Exception as e:
    print(f"Error loading data from JSON file: {e}")

mapping_yago_wiki = {}

for key, value in mapping.items():
    # Remove trailing '>' from key and value
    fixed_key = key.rstrip('>')
    fixed_value = value.rstrip('>')
    
    # Update the updated_mapping_dict with the fixed key-value pair
    mapping_yago_wiki[fixed_key] = fixed_value

In [20]:
from tqdm import tqdm

organization_list = [
    'Airline', 'Consortium', 'Corporation', 'EducationalOrganization', 'CollegeOrUniversity', 'ElementarySchool', 'HighSchool', 
    'MiddleSchool', 'Preschool', 'School', 'FundingScheme', 'GovernmentOrganization', 'LibrarySystem',
    'AnimalShelter', 'ArchiveOrganization', 'AutoBodyShop', 'AutoDealer', 'AutoPartsStore', 'AutoRental', 
    'AutoRepair', 'AutoWash', 'GasStation', 'MotorcycleDealer', 'MotorcycleRepair', 'ChildCare', ' MedicalOrganization', 'Dentist', ' LocalBusiness', 
    'DryCleaningOrLaundry', 'FireStation', 'Hospital', 'PoliceStation', 'EmploymentAgency', 'AdultEntertainment', 
    'AmusementPark', 'ArtGallery', 'Casino', 'ComedyClub', 'MovieTheater', 'NightClub', 'AccountingService', 
    'AutomatedTeller', 'BankOrCreditUnion', 'InsuranceAgency', 'Bakery', 'BarOrPub', 'Brewery', 'CafeOrCoffeeShop', 
    'Distillery', 'FastFoodRestaurant', 'IceCreamShop', 'Restaurant', 'Winery', 'PostOffice', 'BeautySalon', 
    'DaySpa', 'HairSalon', 'HealthClub', 'NailSalon', 'TattooParlor', 'Electrician', 'GeneralContractor', 
    'HVACBusiness', 'HousePainter', 'Locksmith', 'MovingCompany', 'Plumber', 'RoofingContractor', 'InternetCafe', 
    'Attorney', 'Notary', 'Library', 'BedAndBreakfast', 'Campground', 'Hostel', 'Hotel', 'Motel', 'Resort', 'SkiResort',
    'VacationRental', 'DiagnosticLab', 'NGO', 'NewsMediaOrganization', 'OnlineStore', 'DanceGroup', 'MusicGroup', 
    'TheaterGroup', 'PoliticalParty', 'FundingAgency', 'ResearchProject', 'ResearchOrganization', 
    'SearchRescueOrganization', 'SportsTeam', 'WorkersUnion', 'OnlineBusiness', 'PerformingGroup', 'Project', 'SportsOrganization', 
    'MedicalBusiness', 'CommunityHealth', 'Dermatology', 'DietNutrition', 'Emergency', 'Geriatric', 'Gynecologic',
    'MedicalClinic', 'CovidTestingFacility', 'Midwifery', 'Nursing', 'Obstetric', 'Oncologic', 'Optician', 'Optometric',
    'Otolaryngologic', 'Pediatric', 'Pharmacy', 'Physician', 'IndividualPhysician', 'PhysiciansOffice', 'Physiotherapy',
    'PlasticSurgery', 'Podiatric', 'PrimaryCare', 'Psychiatric', 'PublicHealth', 'ProfessionalService', 'RadioStation',
    'RealEstateAgent', 'RecyclingCenter', 'SelfStorage', 'ShoppingCenter', 'SportsActivityLocation', 'BowlingAlley',
    'ExerciseGym', 'GolfCourse', 'PublicSwimmingPool', 'SportsClub', 'StadiumOrArena', 'TennisComplex',
    'Store', 'BikeStore', 'BookStore', 'ClothingStore', 'ComputerStore', 'ConvenienceStore', 'DepartmentStore',
    'ElectronicsStore', 'Florist', 'FurnitureStore', 'GardenStore', 'GroceryStore', 'HardwareStore', 'HobbyShop',
    'HomeGoodsStore', 'JewelryStore', 'LiquorStore', 'MensClothingStore', 'MobilePhoneStore', 'MovieRentalStore',
    'MusicStore', 'OfficeEquipmentStore', 'OutletStore', 'PawnShop', 'PetStore', 'ShoeStore', 'SportingGoodsStore',
    'TireShop', 'ToyStore', 'WholesaleStore', 'TelevisionStation', 'TouristInformationCenter', 'TravelAgency',
    'VeterinaryCare','ResearchProject', 'ResearchOrganization', 'EducationalOrganization', 'Consortium', 'FundingAgency', 'Bank', 
    'BroadcastingCorporation', 'Charity', 'Company', 'ConsultingFirm', 
    'CulturalOrganization', 'EconomicOrganization', 'GovernmentAgency', 'HealthInstitution', 
    'InternationalOrganization', 'LegalOrganization', 'MilitaryOrganization', 'NonProfitOrganization', 
    'PoliticalOrganization', 'ProfessionalOrganization', 'PublishingHouse', 'ReligiousOrganization', 
    'ResearchInstitute', 'SocialOrganization', 'TradeAssociation', 'TransportCompany', 
    'University', 'VentureCapitalFirm'
]


person_list = [
    'Patient', 'Human', 'Artist', 'Athlete', 'Author', 'Composer', 'Inventor', 'Philosopher', 
    'Politician', 'Scientist', 'Actor', 'Musician', 'Teacher', 
    'Doctor', 'Lawyer', 'Entrepreneur', 'Engineer', 'Architect'
]

place_list = [
    'Apartment', 'CampingPitch', 'SingleFamilyResidence', 'HotelRoom', 'MeetingRoom', 'Suite', 'City', 
    'Country', 'SchoolDistrict', 'State', 'Airport', 'Aquarium', 'Beach', 'BoatTerminal', 'Bridge', 'BusStation', 
    'BusStop', 'Campground', 'Cemetery', 'Crematorium', 'EducationalOrganization', 'EventVenue', 'FireStation', 'CityHall', 
    'Courthouse', 'DefenceEstablishment', 'Embassy', 'LegislativeBuilding', 'Hospital', 'MovieTheater', 'Museum', 'MusicVenue', 'Park', 
    'ParkingFacility', 'PerformingArtsTheater', 'PlaceOfWorship', 'BuddhistTemple', 'CatholicChurch', 'HinduTemple', 'Mosque', 'Synagogue', 'Playground', 
    'PoliceStation', 'PublicToilet', 'RVPark', 'StadiumOrArena', 'SubwayStation', 'TaxiStand', 'TrainStation', 'Zoo', 
    'Canal', 'LakeBodyOfWater', 'OceanBodyOfWater', 'Pond', 'Reservoir', 'RiverBodyOfWater', 'SeaBodyOfWater', 
    'Waterfall', 'Continent', 'Mountain', 'Volcano', 'ApartmentComplex', 'GatedResidenceCommunity', 'TouristAttraction', 
    'TouristDestination', 'Accommodation', 'AdministrativeArea', 'CivicStructure', 'Landform', 
    'LandmarksOrHistoricalBuildings', 'LocalBusiness', 'Residence', 'Comune', 'Human_settlement', 'CapitalCity', 'Continent', 'District', 'Island', 'Peninsula', 
    'Desert', 'Forest', 'Glacier', 'MountainRange', 'Valley', 
    'Bay', 'Cape', 'Gulf', 'Harbor', 'Lagoon', 'Strait', 
    'ArchaeologicalSite', 'Castle', 'Fortress', 'HistoricalSite', 'Palace', 
    'Monument', 'Memorial', 'CitySquare', 'Plaza', 'Observatory', 
    'Lighthouse', 'Amphitheater', 'Conservatory', 'Gallery', 'Library'
]



# Initialize the lists and a variable to track the last classified item
ORG = []
LOC = []
PERS = []
OTHERS = []

last_classified_item = None

for line in tqdm(chunk):
    parts = line.split('\t')
    if len(parts) >= 3:
        input_string = parts[2].rsplit('/', 1)[-1]
        cleaned_string = input_string.rstrip('> .')
        item = parts[0].rsplit('/', 1)[-1].rstrip('> .')
        
        # Skip the item if it's the same as the last classified one
        #if item == last_classified_item:
           #continue
        
        if item in set(mapping_yago_wiki.keys()):
            if cleaned_string in organization_list:
                ORG.append(parts[0])
                last_classified_item = item  # Update last classified item
                print(f"ORG: {item} --> {cleaned_string}")
            elif cleaned_string in place_list:
                LOC.append(parts[0])
                last_classified_item = item  # Update last classified item
                print(f"LOC: {item} --> {cleaned_string}")
            elif cleaned_string in person_list:
                PERS.append(parts[0])
                last_classified_item = item  # Update last classified item
                print(f"PERS: {item} --> {cleaned_string}")
            elif cleaned_string != "" and last_classified_item != item:
                OTHERS.append(line)
                last_classified_item = item  # Update last classified item
                print(f"OTHERS: {item} --> {cleaned_string}")
        else:
            print(cleaned_string)


  0%|          | 1/5394049 [00:01<1636:47:42,  1.09s/it]

PERS: Harald_Ringstorff --> Human


  0%|          | 2/5394049 [00:02<1511:55:11,  1.01s/it]

OTHERS: Ecclesiastes --> Religious_text


  0%|          | 4/5394049 [00:03<1425:52:33,  1.05it/s]

...

  0%|          | 115/5394049 [01:46<1469:06:50,  1.02it/s]

OTHERS: Zone_of_the_Enders --> VideoGameSeries
LOC: Chico,_Washington --> AdministrativeArea


  0%|          | 115/5394049 [01:47<1403:54:17,  1.07it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2458/1482596078.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1326, in structured_traceback
    return VerboseTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/

In [5]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round3_2019_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R3_sorted_mentions = json.load(file)

# SPLIT OVER THE QUARTILES

n = len(R3_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = R3_sorted_mentions[:q1_idx]
q2 = R3_sorted_mentions[q1_idx:q2_idx]
q3 = R3_sorted_mentions[q2_idx:q3_idx]
q4 = R3_sorted_mentions[q3_idx:]


sample_size = 1000 
R3_sample_keys = []
R3_sample_keys = R3_sample_keys + random.sample(q1, sample_size)
R3_sample_keys = R3_sample_keys + random.sample(q2, sample_size)
R3_sample_keys = R3_sample_keys + random.sample(q3, sample_size)
R3_sample_keys = R3_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in R3_sample_keys}

# find the mention in the table
tables = "./data/Dataset/Dataset/Round3_2019/tables/"
cta_file = './data/Dataset/Dataset/Round3_2019/gt/CTA_Round3_gt.csv'
os.listdir(tables)


# Apply the function and create the 'key' column
cta_keys = {}
df = pd.read_csv(cta_file, header=None)
category_list = []

for row_idx in range(df.shape[0]):
    col_idx = 2
    while True:
        try:
            if pd.isna(df.iloc[row_idx,col_idx]):
                category_list.append("OTHERS")
                break
            urls = df.iloc[row_idx,col_idx].split(' ')
        except IndexError as e:
            category_list.append("OTHERS")
            break
        
        #print(f"{df.iloc[row_idx,0]}->{cell_urls} @ {row_idx},{col_idx}")
        find = False
        for url in urls:
            type = url.split('/')[-1]            
            if type == "Person":
                category_list.append("PERS")
                find = True
                break
            elif type == "Location":
                category_list.append("LOC")
                find = True
                break
            elif type == "Organisation":
                category_list.append("ORG")
                find = True
                break
        if find:
            break
        
        col_idx += 1



df["category"] = category_list
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

key_to_cell = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                key_to_cell[key] = tmp_value


def get_keys_from_value(d, value):
    keys = [key for key, val in d.items() if val == value]
    return keys[0]

cea_file = './data/Dataset/Dataset/Round3_2019/gt/CEA_Round3_gt_WD.csv'
mentions = {}
chunk_size = 1000
column_names = ["table_name", "row", "col", "url"] 

total_rows = sum(1 for line in open(cea_file)) - 1  # Exclude header
total_iterations = (total_rows + chunk_size - 1) // chunk_size  # Ceiling division to include last chunk

for chunk_cea in tqdm(pd.read_csv(cea_file, chunksize=chunk_size), total=total_iterations):
    chunk_cea.columns = column_names
    for _, row in chunk_cea.iterrows():
        key = f"{row['table_name']} {row['col']}"
        if key in key_to_cell.keys() and row["url"] in q_ids.values():
            data = key_to_cell[key]
            mentions[get_keys_from_value(q_ids, row["url"])] = (row["url"], data)

print("Processing complete.")




100%|██████████| 391/391 [03:46<00:00,  1.73it/s]

Processing complete.


In [6]:
len(mentions)

3998

In [9]:
import pandas as pd
from collections import Counter

rows = []

url = 'https://lamapi.hel.sintef.cloud/entity/labels?token=lamapi_demo_2023'

# Define the headers
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# Iterate over mentions items and collect rows
for text, (url, label) in tqdm(mentions.items(), desc='Processing mentions', unit='item'):
    match = re.search(r'Q(\d+)$', url)
    
    if not match:
        continue
    
    data = {
        'json': [match[0]]
    }

    json_data = json.dumps(data)
    response = requests.post(url, headers=headers, data=json_data)
    
    try:
        desc = response.json()['entities'][match[0]]['descriptions']['en']['value']
        new_row = {'text': text, 'label': label, 'desc': desc}
        rows.append(new_row)
    except:
        pass

# Create DataFrame from the list of rows
df = pd.DataFrame(rows)
category_counts = Counter(df['label'])

# Display the counts for each category
for category, count in category_counts.items():
    print(f'{category}: {count}')
    
df.to_csv('./data/embedding_training_data/Round3_train.csv', index=False)


Processing mentions:  12%|█▏        | 461/3998 [12:45<1:37:56,  1.66s/item]


ConnectionError: HTTPSConnectionPool(host='www.wikidata.org', port=443): Max retries exceeded with url: /entity/Q2734717 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa027057d10>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [ ]:
### in case you want HT2

tables_path = "C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/Dataset/Dataset/HardTablesR2/tables/"
cea_file = 'C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/Dataset/Dataset/HardTablesR2/gt/cea.csv'
os.listdir(tables_path)
# Initialize logging
#logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Read the cea_file and create a key-value dictionary
df = pd.read_csv(cea_file, header=None)
df["key"] = df[0] + " " + df[1].astype(str) + " " + df[2].astype(str)
cea_values_dict = dict(zip(df["key"].values, df[3].values))
cea_keys_set = set(df["key"].values)

# Function to process a single table file
def process_table_file(table_file):
    try:
        table_name = os.path.splitext(os.path.basename(table_file))[0]
        df = pd.read_csv(table_file)
        local_key_to_cell = {}
        
        for row in range(df.shape[0]):
            for col in range(df.shape[1]):
                key = f"{table_name} {row+1} {col}"
                if key in cea_keys_set:
                    cell_value = df.iloc[row, col]
                    local_key_to_cell[key] = (cell_value, cea_values_dict[key])
                    break  # Exit inner loop early as only one match per row/col is needed
        
        return local_key_to_cell
    except Exception as e:
        logging.error(f"Error processing {table_file}: {e}")
        return {}

pattern = r'^\.'

# Create a list of file paths, excluding files that start with a dot
table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path) if not re.match(pattern, table)]

# Process tables sequentially
key_to_cell = {}
for table_file in tqdm(table_files, desc="Processing tables"):
    local_key_to_cell = process_table_file(table_file)
    key_to_cell.update(local_key_to_cell)



with open('C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/work/HT2_ner_type.json', 'r') as f:
    ner_type = json.load(f)

In [70]:
### in case you want R4

with open('C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/work/data/R4_ner_type_new.json', 'r') as f:
    ner_type = json.load(f)

with open('C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/work/R4_key_to_cell.json', 'r') as f:
    key_to_cell = json.load(f)

In [61]:
sample_size = 4000
key_to_cell_sample = dict(random.sample(list(key_to_cell.items()), sample_size))

In [69]:
(ner_type)

{'004JP0KZ 0': 'OTHERS',
 '0051358G 0': 'OTHERS',
 '007O51ZZ 0': 'None',
 '00E5422W 0': 'OTHERS',
 '00E5422W 1': 'OTHERS',
 '00E5422W 2': 'PERS',
 '00E5422W 3': 'None',
 '00ECUL14 0': 'OTHERS',
 '00GX3RE0 0': 'OTHERS',
 '00KVC61H 0': 'OTHERS',
 '00P63AAJ 0': 'None',
 '00P63AAJ 1': 'OTHERS',
 '00P63AAJ 4': 'PERS',
 '00TWHQA3 0': 'None',
 '00UQHNO3 0': 'OTHERS',
 '00YF468U 0': 'None',
 '0172127I 0': 'OTHERS',
 '01ANSETB 0': 'None',
 '01ANSETB 1': 'None',
 '01CYBJ7I 0': 'None',
 '01DQQLCP 0': 'OTHERS',
 '01DQQLCP 4': 'OTHERS',
 '01E31BCQ 0': 'None',
 '01EUFAWG 0': 'None',
 '01GQNYIT 0': 'None',
 '01IMPH1S 0': 'None',
 '01LAC0PK 0': 'OTHERS',
 '01LHSIJE 0': 'None',
 '01MBA5Y8 0': 'OTHERS',
 '01Q0EMNA 0': 'OTHERS',
 '01Q0EMNA 1': 'OTHERS',
 '01RLDR8E 0': 'None',
 '01SZ0JR8 0': 'OTHERS',
 '01U1PQIC 0': 'None',
 '01VFNPOS 0': 'OTHERS',
 '01VFNPOS 4': 'None',
 '01WL7IBA 0': 'None',
 '01WRLMW1 0': 'OTHERS',
 '01WRLMW1 2': 'None',
 '01ZJDM98 0': 'None',
 '01ZJDM98 3': 'None',
 '01ZJH3QE 0': 'Non

In [76]:
set(ner_type.values())

{'LOC', None, 'ORG', 'OTHERS', 'PERS'}

In [84]:
def get_query(name, value):
    name = str(name).replace('"', ' ')
    if value is not None:
        # Soft filtering constraint
        query_dict = {
            "query": {
                "bool": {
                    "must": [
                        {"match": {"name": {"query": name, "boost": 2.0}}}
                    ],
                    "should": [
                        {"term": {"NERtype": value}}
                    ]
                }
            }
        }
        params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': json.dumps(query_dict),  # Convert the query dictionary to a JSON string
            'sort': [
                '{"popularity": {"order": "desc"}}'
            ]
        }
    
    return params

url = 'https://lamapi.hel.sintef.cloud/entity/labels?token=lamapi_demo_2023'

# Define the headers
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

queries = []
for key in tqdm(key_to_cell_sample):
    id_table, _, id_col = key.split(" ")
    name = key_to_cell[key][0]
    q_ids = key_to_cell[key][1].split(' ')
    new_key = f"{id_table} {id_col}"
    if new_key in ner_type:
        NER_type = ner_type[new_key]
        if NER_type != None:
            query = get_query(name, NER_type)
            
            matched_results = []
            for q_id in q_ids:            
                match = re.search(r'Q(\d+)$', q_id)
                if not match:
                    continue
                data = {
                    'json': [match[0]]
                }

                json_data = json.dumps(data)
                response = requests.post(url, headers=headers, data=json_data)
                if len(response.json()) == 0:
                    continue
                
                true_ner = response.json()[match[0]]['NERtype']
                break

            if match:
                data = json.loads(query['query'])
                ner_type_list = data['query']['bool']['should'][0]['term']['NERtype']
                # ner_type_list is the ner column
                queries.append((query, match[0],ner_type_list, true_ner))


  1%|          | 27/4000 [00:05<12:39,  5.23it/s]


KeyboardInterrupt: 

In [1]:
def get_query(name, value):
    name = str(name).replace('"', ' ')

    if value is not None:
        # Hard filtering constraint
        query_dict = {
            "query": {
                "bool": {
                    "must": [
                        {"match": {"name": {"query": name, "boost": 2.0}}},
                        {"term": {"NERtype": value}}
                    ]
                }
            }
        }
        params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': json.dumps(query_dict),  # Convert the query dictionary to a JSON string
            'sort': [
                '{"popularity": {"order": "desc"}}'
            ]
        }    

    return params

url = 'https://lamapi.hel.sintef.cloud/entity/labels?token=lamapi_demo_2023'

# Define the headers
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

queries = []
for key in tqdm(key_to_cell_sample):
    id_table, _, id_col = key.split(" ")
    name = key_to_cell[key][0]
    q_ids = key_to_cell[key][1].split(' ')
    new_key = f"{id_table} {id_col}"
    if new_key in ner_type:
        NER_type = ner_type[new_key]
        if NER_type is None:
            print(f"q_ids: {q_ids}, ner_type key: {new_key}")
        query = get_query(name, NER_type)
        
        matched_results = []
        for q_id in q_ids:
            match = re.search(r'Q(\d+)$', q_id)
            if not match:
                continue
            data = {
                'json': [match[0]]
            }

            json_data = json.dumps(data)
            response = requests.post(url, headers=headers, data=json_data)
            if len(response.json()) == 0:
                continue
            
            true_ner = response.json()[match[0]]['NERtype']
            break

        if match:
            data = json.loads(query['query'])
            ner_type_list = data['query']['bool']['must'][1]['term']['NERtype']
            queries.append((query, match[0],ner_type_list,true_ner))


NameError: name 'tqdm' is not defined

In [87]:
import aiohttp
import asyncio
import backoff
import nest_asyncio
import random
from tqdm import tqdm
import numpy as np

# Assume queries is a list of tuples [(param1, id1), (param2, id2), ...]

failed_queries = {}
url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo, 
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError), 
    max_tries=10, 
    max_time=400
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        # Convert all params to str, int, or float
        #params = {k: (int(v) if isinstance(v, np.integer) else str(v)) for k, v in params.items()}
        async with session.get(url, params=params, headers=headers, timeout=50) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []

async def process_item(session, url, id, headers, params, semaphore, pbar):
    try:
        data = await fetch(session, url, params, headers, semaphore)
    except aiohttp.ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{id}'")
            asyncio.get_event_loop().call_soon_threadsafe(pbar.update, 1)
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        for item in data:
            if id == item.get('id'):
                asyncio.get_event_loop().call_soon_threadsafe(pbar.update, 1)
                pos_score = item.get('pos_score', 0)
                if pos_score:
                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                else:
                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                return mrr_increment, 1

    return 0, 0

async def main(queries, url, pbar, failed_queries):
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0

    async with aiohttp.ClientSession() as session:
        tasks = []
        for param, id, _, _ in queries:
            tasks.append(process_item(session, url, id, headers, param, semaphore, pbar))
        
        results = await asyncio.gather(*tasks)
        
        for (mrr_increment, count), (param, id, col_NERtype, item_NERtype) in zip(results, queries):
            if mrr_increment == 0 and count == 0:
                failed_queries[id] = (col_NERtype, item_NERtype)
            else:
                m_mrr += mrr_increment
                cont_el += count

        asyncio.get_event_loop().call_soon_threadsafe(pbar.close)

    print(f"Coverage of 2T: {cont_el / len(queries)}")
    print(f"Measure Reciprocal Rank of 2T: {m_mrr / len(queries)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(queries))
        asyncio.run(main(queries, url, pbar, failed_queries))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(queries, url, pbar, failed_queries))



  0%|          | 0/506 [00:00<?, ?it/s]

In [82]:
with open('C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/R4_failed_queries_HARD.json', 'w') as json_file:
    json.dump(failed_queries, json_file, indent=4)

In [44]:
failed_queries

{'Q41319588': 'OTHERS',
 'Q279292': 'OTHERS',
 'Q306216': 'LOC',
 'Q64616170': 'ORG',
 'Q105231773': 'ORG',
 'Q2201159': 'OTHERS',
 'Q176830': 'LOC',
 'Q10333266': 'LOC',
 'Q105064589': 'ORG',
 'Q1570564': 'OTHERS',
 'Q56477536': 'ORG',
 'Q1319364': 'OTHERS',
 'Q1168148': 'OTHERS',
 'Q84933431': 'OTHERS',
 'Q83640976': 'OTHERS',
 'Q60372364': 'OTHERS',
 'Q11365639': 'LOC',
 'Q89692143': 'ORG',
 'Q2823425': 'LOC',
 'Q18845117': 'OTHERS',
 'Q41319497': 'OTHERS',
 'Q9282293': 'LOC',
 'Q57033626': 'OTHERS',
 'Q889587': 'OTHERS',
 'Q2153640': 'OTHERS',
 'Q1646461': 'LOC',
 'Q18882447': 'OTHERS',
 'Q1547773': 'ORG',
 'Q14217100': 'LOC',
 'Q64682396': 'OTHERS',
 'Q96069725': 'LOC',
 'Q12544': 'LOC',
 'Q5970': 'ORG',
 'Q86354019': 'OTHERS',
 'Q4651363': 'ORG',
 'Q311695': 'OTHERS',
 'Q93040237': 'OTHERS',
 'Q41796518': 'OTHERS',
 'Q278151': 'OTHERS',
 'Q2981963': 'LOC',
 'Q80111478': 'OTHERS',
 'Q91511110': 'OTHERS',
 'Q834482': 'OTHERS',
 'Q86248958': 'OTHERS',
 'Q2272973': 'OTHERS',
 'Q34293

In [45]:
for p, k, _ in queries:
    if k == "Q41319588":
        print(p)

{'name': 'Bidstock', 'token': 'lamapi_demo_2023', 'kg': 'wikidata', 'limit': 1000, 'query': '{"query": {"bool": {"must": [{"match": {"name": {"query": "Bidstock", "boost": 2.0}}}, {"term": {"NERtype": "OTHERS"}}]}}}', 'sort': ['{"popularity": {"order": "desc"}}']}


In [2]:
####################
# READ THE JSON
#####################

json_file_path = "C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/Round1_T2D_f3_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R1_sorted_mentions = json.load(file)


# SPLIT OVER THE QUARTILES

n = len(R1_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = R1_sorted_mentions[:q1_idx]
q2 = R1_sorted_mentions[q1_idx:q2_idx]
q3 = R1_sorted_mentions[q2_idx:q3_idx]
q4 = R1_sorted_mentions[q3_idx:]

sample_size = 1000
R1_sample_keys = []
R1_sample_keys = R1_sample_keys + random.sample(q1, sample_size)
R1_sample_keys = R1_sample_keys + random.sample(q2, sample_size)
R1_sample_keys = R1_sample_keys + random.sample(q3, sample_size)
R1_sample_keys = R1_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in R1_sample_keys}

In [4]:
# find the mention in the table
tables = "C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/Dataset/Dataset/Round1_T2D/tables/"
cta_file = 'C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/Dataset/Dataset/Round1_T2D/gt/CTA_Round1_gt.csv'
os.listdir(tables)

mapping = {
    "LOC": [
        "Place", "PopulatedPlace", "City", "Country", "Region", "Mountain", "Island", "Lake", "River",
        "Park", "Building", "HistoricPlace", "Monument", "Bridge", "Road", "Airport"
    ],
    "PERS": [
        "Person", "Artist", "Athlete", "Politician", "Scientist", "Writer", "Actor", "Musician", "MilitaryPerson",
        "Religious", "Royalty", "Criminal"
    ],
    "ORG": [
        "Organisation", "Company", "EducationalInstitution", "PoliticalParty", "SportsTeam", "Non-ProfitOrganisation",
        "GovernmentAgency", "ReligiousOrganisation", "Band", "Library", "Museum", "Hospital", "University", "TradeUnion"
    ]
}

# Create reverse mapping
reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Define function to map df[2] values to their categories
def map_class_to_category(class_name):
    return reverse_mapping.get(class_name, "OTHERS")

# Apply the function and create the 'key' column
cta_keys = {}
df = pd.read_csv(cta_file, header=None)
type = df[2].astype(str).str.split('/').str[-1]
df["category"] = type.apply(map_class_to_category)
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

key_to_cell = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                key_to_cell[key] = tmp_value

100%|██████████| 64/64 [00:02<00:00, 30.33it/s]


In [6]:
def get_keys_from_value(d, value):
    keys = [key for key, val in d.items() if val == value]
    return keys[0]

In [7]:
cea_file = 'C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/Dataset/Dataset/Round1_T2D/gt/CEA_Round1_gt_WD.csv'
mentions = {}
chunk_size = 1000
column_names = ["table_name", "row", "col", "url"] 

total_rows = sum(1 for line in open(cea_file)) - 1  # Exclude header
total_iterations = (total_rows + chunk_size - 1) // chunk_size  # Ceiling division to include last chunk

count = 0
for chunk_cea in tqdm(pd.read_csv(cea_file, chunksize=chunk_size), total=total_iterations):
    chunk_cea.columns = column_names
    for _, row in chunk_cea.iterrows():
        key = f"{row['table_name']} {row['col']}"
        if key in key_to_cell.keys() and row["url"] in q_ids.values():
            count += 1
            data = key_to_cell[key]
            mentions[get_keys_from_value(q_ids, row["url"])] = (row["url"], data)

print("Processing complete.")

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:04<00:00,  2.17it/s]

Processing complete.


In [10]:
url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
failed_queries = {}

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo, 
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError), 
    max_tries=5, 
    max_time=300
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        async with session.get(url, params=params, headers=headers, timeout=30) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []
async def process_item(session, name, value, url, headers, semaphore, pbar):
    ### SOFT FILTERING CONTSTRAINT
    params = {
        'name': name,
        'token': 'lamapi_demo_2023',
        'kg': 'wikidata',
        'limit': 1000,
        'query': f'''
            {{
                "query": {{
                    "bool": {{
                        "must": [
                            {{
                                "match": {{
                                    "name": {{
                                        "query": "{name}",
                                        "boost": 2.0
                                    }}
                                }}
                            }}
                        ],
                        "should": [
                            {{
                                "term": {{
                                    "NERtype": "{value[1]}"
                                }}
                            }}
                        ]
                    }}
                }}
            }}
            ''',
        'sort': [
            f'''{{"popularity": {{"order": "desc"}}}}'''
        ]
    }



    try:
        data = await fetch(session, url, params, headers, semaphore)
    except ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{name}'")
            pbar.update(1)  # No need to await here
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        pbar.update(1)  # No need to await here
        for item in data:
            GT_id_match = re.search(r'Q(\d+)$', value[0])

            if GT_id_match:
                GT_id = GT_id_match[0]
                if GT_id == item.get('id'):
                    pos_score = item.get('pos_score', 0)
                    if pos_score:
                        mrr_increment = (num_result - (pos_score * num_result)) / num_result
                    else:
                        mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                    return mrr_increment, 1

        print(f"{name}: {GT_id_match[0]} NOT FOUND in {value[1]}")
        print("___________________________")

    return 0, 0

async def main(mentions, url, pbar, failed_queries):
    string_name_list = mentions
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0
    async with aiohttp.ClientSession() as session:
        tasks = []
        for name, type in string_name_list.items():
            tasks.append(process_item(session, name, type, url, headers, semaphore, pbar))
        
        results = await asyncio.gather(*tasks)
        
        for (mrr_increment, count), (name, url_id) in zip(results, string_name_list.items()):
            if mrr_increment == 0 and count == 0:                
                failed_queries[url_id] = name
                            
            m_mrr += mrr_increment
            cont_el += count

        pbar.close()  # No need to await here

    print(f"Coverage of R1: {cont_el / len(mentions)}")
    print(f"Measure Reciprocal Rank of R1: {m_mrr / len(mentions)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(mentions))
        asyncio.run(main(mentions, url, pbar, failed_queries))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(mentions, url, pbar))


  1%|          | 48/3865 [00:37<31:35,  2.01it/s]  

st helena pounds: Q374453 NOT FOUND in OTHERS
___________________________


  2%|▏         | 63/3865 [00:45<29:30,  2.15it/s]

abraham the syrian: Q1292819 NOT FOUND in OTHERS
___________________________


  5%|▍         | 189/3865 [01:50<12:25,  4.93it/s]  

...

southeastern five-lined skink: Q137678 NOT FOUND in OTHERS
___________________________


 99%|█████████▉| 3839/3865 [11:55<00:03,  6.72it/s]

matrix, the: Q83495 NOT FOUND in OTHERS
___________________________


100%|█████████▉| 3862/3865 [11:58<00:00,  5.38it/s]

Coverage of R1: 0.9844760672703752
Measure Reciprocal Rank of R1: 0.9581293661060406


In [52]:
with open('C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/HT2_failed_queries_SOFT.json', 'w') as json_file:
    json.dump(failed_queries, json_file, indent=4)

In [52]:
with open('C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/HT2_failed_queries_HARD.json', 'r') as f:
    failed_queries_hard = json.load(f)

with open('C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/HT2_failed_queries_SOFT.json', 'r') as f:
    failed_queries_soft = json.load(f)

In [48]:
(set(failed_queries_hard.keys()) & set(failed_queries_soft.keys()))

{'Q102435',
 'Q1205642',
 'Q12722',
 'Q13440',
 'Q134735',
 'Q1353876',
 'Q1364541',
 'Q1423078',
 'Q15814081',
 'Q15856153',
 'Q15989',
 'Q16511',
 'Q1775206',
 'Q1779859',
 'Q1787505',
 'Q18845586',
 'Q18882458',
 'Q18888595',
 'Q19362881',
 'Q19381330',
 'Q200634',
 'Q20805463',
 'Q21560198',
 'Q2162084',
 'Q2395500',
 'Q276837',
 'Q282269',
 'Q2908282',
 'Q3231617',
 'Q3500538',
 'Q38049003',
 'Q3917337',
 'Q4529416',
 'Q59311769',
 'Q612773',
 'Q62017162',
 'Q65772024',
 'Q66623013',
 'Q6819071',
 'Q74337267',
 'Q74651710',
 'Q75699321',
 'Q77307090',
 'Q77980182',
 'Q77980709',
 'Q78191247',
 'Q81132',
 'Q834482',
 'Q91330689',
 'Q91511110',
 'Q9336171',
 'Q93725704'}

In [53]:
print(f"failed_queries_hard: {len(failed_queries_hard)} vs failed_queries_soft: {len(failed_queries_soft)}")

failed_queries_hard: 499 vs failed_queries_soft: 358


In [54]:
# 19 entities doesn’t match due to the Hard filtering constraint

missing_values = set(failed_queries_hard) - set(failed_queries_soft)
print(len(missing_values))

143


In [57]:
failed_queries_hard['Q102469']

'LOC'

In [55]:
missing_values

{'Q1013478',
 'Q1022366',
 'Q102469',
 'Q102472401',
 'Q1026019',
 'Q1030389',
 'Q10333266',
 'Q104147571',
 'Q10451',
 'Q104914792',
 'Q105064589',
 'Q105231773',
 'Q1112438',
 'Q11352733',
 'Q11353619',
 'Q11511194',
 'Q1205685',
 'Q1218597',
 'Q12544',
 'Q1319988',
 'Q142636',
 'Q1445363',
 'Q14875596',
 'Q1489862',
 'Q150489',
 'Q1530878',
 'Q1547773',
 'Q1550449',
 'Q1559811',
 'Q1569748',
 'Q1569781',
 'Q1569821',
 'Q1569868',
 'Q1570564',
 'Q15708337',
 'Q1581162',
 'Q1630016',
 'Q163164',
 'Q1634594',
 'Q164596',
 'Q16735627',
 'Q1746330',
 'Q1759911',
 'Q1803466',
 'Q19375324',
 'Q194104',
 'Q2079479',
 'Q215819',
 'Q2201159',
 'Q2209140',
 'Q223799',
 'Q224494',
 'Q2272973',
 'Q2274963',
 'Q2299611',
 'Q2375620',
 'Q2442235',
 'Q2534226',
 'Q2591630',
 'Q2616970',
 'Q2616988',
 'Q2617303',
 'Q2620426',
 'Q2641647',
 'Q2652121',
 'Q267144',
 'Q2673008',
 'Q26786604',
 'Q2739939',
 'Q2753179',
 'Q278151',
 'Q278877',
 'Q2862516',
 'Q2894053',
 'Q2982552',
 'Q2987303',
 'Q306241